In [1]:
!pip install qiskit qiskit-machine-learning qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [3]:
data = load_breast_cancer()
X = data.data[:, :2]
y = data.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

feature_map = ZZFeatureMap(feature_dimension=30, reps=1)
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

In [5]:
qsvm = SVC(kernel=quantum_kernel.evaluate)

qsvm.fit(X_train_scaled, y_train)
y_pred = qsvm.predict(X_test_scaled)
accuracy = qsvm.score(X_test_scaled, y_test)
print(f"Quantum SVM Accuracy: {accuracy:.2f}")


Quantum SVM Accuracy: 0.56


In [6]:
import numpy as np

In [7]:
def predict_quantum_breast_cancer(features_str):
    try:

        cleaned_features = [
            float(x.strip().replace('\n', '').replace('\r', ''))
            for x in features_str.strip().split(',')
            if x.strip() != ''
        ]

        if len(cleaned_features) != 2:
            return "Please enter exactly 2 comma-separated values (features)."

        input_scaled = scaler.transform([cleaned_features])
        prediction = qsvm.predict(input_scaled)[0]

        return "Benign" if prediction == 1 else "Malignant"

    except Exception as e:
        return f"Error while processing input: {str(e)}"


In [8]:
ftrs = input("Please enter 2 values (comma, serperable) : ")
print(f"The prediction is : {predict_quantum_breast_cancer(ftrs)}")

Please enter 2 values (comma, serperable) : 2.3, 4.5
The prediction is : Benign


### Cardio-Vascular Detection

In [9]:
!pip install qiskit qiskit-aer qiskit-machine-learning scikit-learn pandas --quiet

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [11]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes

In [12]:
from qiskit_aer import Aer
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import COBYLA

In [13]:
import pandas as pd
data = pd.read_csv('/content/heart.csv')

In [14]:
X = data.drop('target', axis=1).values
y = data['target'].values

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
np.random.seed(42)

In [18]:
feature_map = ZZFeatureMap(feature_dimension=2, reps=2, entanglement='linear')
ansatz = RealAmplitudes(num_qubits=2, reps=2, entanglement='linear')
optimizer = COBYLA(maxiter=100)
backend = Aer.get_backend("aer_simulator_statevector")

In [19]:
from qiskit.primitives import Sampler
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms.classifiers import VQC

In [20]:
sampler = AerSampler()

vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

vqc.fit(X_train, y_train)

In [21]:
y_pred = vqc.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['No Disease', 'Disease']))

              precision    recall  f1-score   support

  No Disease       0.44      0.41      0.43        29
     Disease       0.50      0.53      0.52        32

    accuracy                           0.48        61
   macro avg       0.47      0.47      0.47        61
weighted avg       0.47      0.48      0.47        61



In [22]:
def predict_heart_disease(age, sex, cp, trestbps, chol, fbs, restecg,
                          thalach, exang, oldpeak, slope, ca, thal):
  try:

    user_input = np.array([[age, sex, cp, trestbps, chol, fbs, restecg,
                            thalach, exang, oldpeak, slope, ca, thal]])

    user_scaled = scaler.transform(user_input)
    user_pca = pca.transform(user_scaled)

    prediction = int(vqc.predict(user_pca))
    return "Disease Detected" if prediction == 1 else "No Disease"

  except Exception as e:
    return f"Error: {str(e)}"

In [23]:
predict_heart_disease(63,1,3,150,233,1,0,159,0,1.4,0,0,1)

'No Disease'